In [10]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [6]:
death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 1,
                               22: 3, 23: 3, 24: 2, 25: 5, 26: 2,
                               27: 7, 28: 4, 29: 21, 30: 11,
                               31: 10, 32: 23, 33: 10, 34: 25,
                               35: 19, 36: 25, 37: 14, 38: 20,
                               39: 28, 40: 14, 41: 19, 42: 5})

In [7]:
last_day = max(death_dict.keys())

In [8]:
index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + last_day)]
deaths =[death_dict[day - 28] for day in range(29 + last_day)]

In [11]:
death_df = pd.DataFrame(deaths, index=index, columns=['deaths'])

In [164]:
death_df.to_pickle('../data/chi_deaths.pickle')